# Task 2

## Chapter 1: Initialisation and imports

In [1]:
from pyspark import SparkConf, SparkContext
import itertools
import time
from colorama import Fore

import warnings
warnings.filterwarnings('ignore')

## Chapter 2: Utility functions

### Mining based utility functions

In [2]:
def create_count_dict(items, support_value):
    item_counts = {}
    for item in items:
        if item not in item_counts.keys():
            item_counts[item] = 1
        else:
            if item_counts[item] < support_value:
                item_counts[item] = item_counts[item] + 1
    return item_counts


def create_count_dict_tuple(chunk, support_value, candidate_tuples):
    tuple_counts = {}
    for basket in chunk:
        for candidate in candidate_tuples:
            if set(candidate).issubset(basket):
                if candidate in tuple_counts and tuple_counts[candidate] < support_value:
                    tuple_counts[candidate] += 1
                elif candidate not in tuple_counts:
                    tuple_counts[candidate] = 1
    return tuple_counts


def filter_by_support(count_dict, support_value, is_tuple):
    frequent_candidates = []
    for candidate, count in count_dict.items():
        if count >= support_value:
            frequent_candidates.append(candidate)
            phase1_candidates.append((candidate if is_tuple else tuple({candidate}), 1))
    return frequent_candidates


def get_frequent_candidate_tuples(frequent_items, size):
    candidates = []
    for item_x in frequent_items:
        for item_y in frequent_items:
            combined_set = tuple(sorted(set(item_x + item_y)))
            if len(combined_set) == size:
                if combined_set not in candidates:
                    previous_candidates = list(itertools.combinations(combined_set, size - 1))
                    flag = True
                    for candidate in previous_candidates:
                        if candidate not in frequent_items:
                            flag = False
                            break
                    if flag:
                        candidates.append(combined_set)
    return candidates


def find_frequent_candidates(chunk, chunk_support, frequent_items, size):
    if size == 2:
        candidates = list(itertools.combinations(sorted(frequent_items), 2))
    else:
        candidates = get_frequent_candidate_tuples(frequent_items, size)
    return filter_by_support(create_count_dict_tuple(chunk, chunk_support, candidates), chunk_support, True)


def apriori(baskets):
    chunk = list(baskets)
    chunk_support = support * (len(chunk) / basket_count)
    items = []
    for basket in chunk:
        for item in basket:
            items.append(item)

    #frequent itemsets of size 1
    frequent_items = filter_by_support(create_count_dict(items, chunk_support), chunk_support, False)
    size = 2
    while len(frequent_items) != 0:
        frequent_items = find_frequent_candidates(chunk, chunk_support, frequent_items, size)
        size += 1

    return [phase1_candidates]

def son(candidate):
    frequent_itemset_count = {}
    for basket in basket_list:
        if isinstance(candidate, tuple):
            if set(candidate).issubset(basket):
                if candidate in frequent_itemset_count and frequent_itemset_count[candidate] < support:
                    frequent_itemset_count[candidate] += 1
                elif candidate not in frequent_itemset_count:
                    frequent_itemset_count[candidate] = 1
        else:
            frequent_itemset_count = create_count_dict(basket, support)

    frequent_itemsets_actual = []
    for itemset, count in frequent_itemset_count.items():
        frequent_itemsets_actual.append((itemset, count))
    return frequent_itemsets_actual

### I/O based utility functions

In [3]:
def get_required_bucket(row):
    line = row.split(",")
    return line[0], line[1]


def write_to_file(candidates, frequent_itemsets, output_file):
    with open(output_file, 'w') as file:
        file.write("Candidates:\n" + candidates + "\n\nFrequent Itemsets:\n" + frequent_itemsets)


def get_output_string(input_tuple, size, output):
    if len(input_tuple) > size:
        output = output[:-1] + "\n\n"

    output = output + "('" + str(input_tuple[0]) + "')," if len(input_tuple) == 1 else output + str(input_tuple) + ","
    return output


def get_output(phase_rdd, is_phase_2):
    size = 1
    output = ""
    sorted_list = sorted(sorted(phase_rdd if is_phase_2 else phase_rdd.map(lambda x: x[0]).collect()), key=len)
    for x in sorted_list:
        output = get_output_string(x, size, output)
        size = len(x)
    return output

## Chapter 3: Datasets chosen

In [4]:
dataset_metadata = {
    'amazon-reviews': {
        'path': './data/amazon-reviews/transactions.csv',
        'output_dir': './logs/amazon-reviews/',
        'minsup': [5000, 3000, 1500, 1000],
    },
    
    
    'groceries': {
        'path': './data/groceries/transactions.csv',
        'output_dir': './logs/groceries/',
        'column_names': 'userId,itemName',
        'minsup': [250, 200, 150, 100, 50],
    },
    
    
    'movielens': {
        'path': './data/movielens/transactions.csv',
        'output_dir': './logs/movielens/',
        'column_names': 'userId,movieId',
        'minsup': [300, 250, 200, 150, 100],
    }
}

In [5]:
dataset_name = 'movielens'

In [6]:
input_filepath = dataset_metadata[dataset_name]['path']
output_dir = dataset_metadata[dataset_name]['output_dir']
minsup_values_list = dataset_metadata[dataset_name]['minsup']
column_names = dataset_metadata[dataset_name]['column_names']

print(minsup_values_list)

[300, 250, 200, 150, 100]


In [7]:
start = time.time()

phase1_candidates = []
support = minsup_values_list[0]

print(Fore.WHITE + f'Running SON Apriori Algorithm on', Fore.GREEN + f'{dataset_name}', 
        Fore.WHITE + f'with minsup value: {support}\n')


spark_configuration = SparkConf().setAppName("cz4032_task2").setMaster('local[*]')
sc = SparkContext(conf=spark_configuration)
sc.setLogLevel("ERROR")

try:
    rdd = sc.textFile(input_filepath)
    

    basket_rdd = rdd.filter(
        lambda x: x != column_names).map(
            lambda x: get_required_bucket(x)
            ).groupByKey().mapValues(set).map(
                lambda x: x[1]
                ).persist()
        
    basket_list = basket_rdd.collect()
    basket_count = len(basket_list)
    
    phase1_map = basket_rdd.mapPartitions(apriori).flatMap(lambda x: x)
    phase1_reduce = phase1_map.reduceByKey(lambda x, y: x + y).persist()

    phase2_map = phase1_reduce.map(lambda x: son(x[0])).flatMap(lambda x: x)
    phase2_reduce = phase2_map.filter(lambda x: x[1] >= support).map(lambda x: x[0]).collect()

    phase1_output = get_output(phase1_reduce, False)[:-1]
    phase2_output = get_output(phase2_reduce, True)[:-1]

    sc.stop()
    
    output_filepath = f'{output_dir}{dataset_name}_minsup_{support}.txt'
    write_to_file(phase1_output, phase2_output, output_filepath)
    
    print(Fore.BLUE + f'--'*30)
    print(Fore.WHITE + f"| MINSUP:",Fore.GREEN + f"{support}")
    print(Fore.WHITE + f"| Time taken:", Fore.GREEN + f"{round(time.time() - start, 1)} seconds.")
    print(Fore.WHITE + f"| Outputs saved to", Fore.GREEN +  f"{output_filepath}.")
    print(Fore.BLUE + f'--'*30)
    
except Exception as e:
    sc.stop()
    print(Fore.RED + f'Spark Server stopped due to exception.')
    print(Fore.RED + f'Error: {e}')

Running SON Apriori Algorithm on movielens with minsup value: 300



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 04:34:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


------------------------------------------------------------
| MINSUP: 300
| Time taken: 10.2 seconds.
| Outputs saved to ./logs/movielens/movielens_minsup_300.txt.
------------------------------------------------------------


In [7]:
start = time.time()

phase1_candidates = []
support = minsup_values_list[1]

print(Fore.WHITE + f'Running SON Apriori Algorithm on', Fore.GREEN + f'{dataset_name}', 
        Fore.WHITE + f'with minsup value: {support}\n')


spark_configuration = SparkConf().setAppName("cz4032_task2").setMaster('local[*]')
sc = SparkContext(conf=spark_configuration)
sc.setLogLevel("ERROR")

try:
    rdd = sc.textFile(input_filepath)
    

    basket_rdd = rdd.filter(
        lambda x: x != column_names).map(
            lambda x: get_required_bucket(x)
            ).groupByKey().mapValues(set).map(
                lambda x: x[1]
                ).persist()
        
    basket_list = basket_rdd.collect()
    basket_count = len(basket_list)
    
    phase1_map = basket_rdd.mapPartitions(apriori).flatMap(lambda x: x)
    phase1_reduce = phase1_map.reduceByKey(lambda x, y: x + y).persist()

    phase2_map = phase1_reduce.map(lambda x: son(x[0])).flatMap(lambda x: x)
    phase2_reduce = phase2_map.filter(lambda x: x[1] >= support).map(lambda x: x[0]).collect()

    phase1_output = get_output(phase1_reduce, False)[:-1]
    phase2_output = get_output(phase2_reduce, True)[:-1]

    sc.stop()
    
    output_filepath = f'{output_dir}{dataset_name}_minsup_{support}.txt'
    write_to_file(phase1_output, phase2_output, output_filepath)
    
    print(Fore.BLUE + f'--'*30)
    print(Fore.WHITE + f"| MINSUP:",Fore.GREEN + f"{support}")
    print(Fore.WHITE + f"| Time taken:", Fore.GREEN + f"{round(time.time() - start, 1)} seconds.")
    print(Fore.WHITE + f"| Outputs saved to", Fore.GREEN +  f"{output_filepath}.")
    print(Fore.BLUE + f'--'*30)
    
except Exception as e:
    sc.stop()
    print(Fore.RED + f'Spark Server stopped due to exception.')
    print(Fore.RED + f'Error: {e}')

Running SON Apriori Algorithm on movielens with minsup value: 250



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 04:35:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


------------------------------------------------------------
| MINSUP: 250
| Time taken: 10.0 seconds.
| Outputs saved to ./logs/movielens/movielens_minsup_250.txt.
------------------------------------------------------------


In [7]:
start = time.time()

phase1_candidates = []
support = minsup_values_list[2]

print(Fore.WHITE + f'Running SON Apriori Algorithm on', Fore.GREEN + f'{dataset_name}', 
        Fore.WHITE + f'with minsup value: {support}\n')


spark_configuration = SparkConf().setAppName("cz4032_task2").setMaster('local[*]')
sc = SparkContext(conf=spark_configuration)
sc.setLogLevel("ERROR")

try:
    rdd = sc.textFile(input_filepath)
    

    basket_rdd = rdd.filter(
        lambda x: x != column_names).map(
            lambda x: get_required_bucket(x)
            ).groupByKey().mapValues(set).map(
                lambda x: x[1]
                ).persist()
        
    basket_list = basket_rdd.collect()
    basket_count = len(basket_list)
    
    phase1_map = basket_rdd.mapPartitions(apriori).flatMap(lambda x: x)
    phase1_reduce = phase1_map.reduceByKey(lambda x, y: x + y).persist()

    phase2_map = phase1_reduce.map(lambda x: son(x[0])).flatMap(lambda x: x)
    phase2_reduce = phase2_map.filter(lambda x: x[1] >= support).map(lambda x: x[0]).collect()

    phase1_output = get_output(phase1_reduce, False)[:-1]
    phase2_output = get_output(phase2_reduce, True)[:-1]

    sc.stop()
    
    output_filepath = f'{output_dir}{dataset_name}_minsup_{support}.txt'
    write_to_file(phase1_output, phase2_output, output_filepath)
    
    print(Fore.BLUE + f'--'*30)
    print(Fore.WHITE + f"| MINSUP:",Fore.GREEN + f"{support}")
    print(Fore.WHITE + f"| Time taken:", Fore.GREEN + f"{round(time.time() - start, 1)} seconds.")
    print(Fore.WHITE + f"| Outputs saved to", Fore.GREEN +  f"{output_filepath}.")
    print(Fore.BLUE + f'--'*30)
    
except Exception as e:
    sc.stop()
    print(Fore.RED + f'Spark Server stopped due to exception.')
    print(Fore.RED + f'Error: {e}')

Running SON Apriori Algorithm on movielens with minsup value: 200



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 04:36:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


------------------------------------------------------------
| MINSUP: 200
| Time taken: 10.8 seconds.
| Outputs saved to ./logs/movielens/movielens_minsup_200.txt.
------------------------------------------------------------


In [7]:
start = time.time()

phase1_candidates = []
support = minsup_values_list[3]

print(Fore.WHITE + f'Running SON Apriori Algorithm on', Fore.GREEN + f'{dataset_name}', 
        Fore.WHITE + f'with minsup value: {support}\n')


spark_configuration = SparkConf().setAppName("cz4032_task2").setMaster('local[*]')
sc = SparkContext(conf=spark_configuration)
sc.setLogLevel("ERROR")

try:
    rdd = sc.textFile(input_filepath)
    

    basket_rdd = rdd.filter(
        lambda x: x != column_names).map(
            lambda x: get_required_bucket(x)
            ).groupByKey().mapValues(set).map(
                lambda x: x[1]
                ).persist()
        
    basket_list = basket_rdd.collect()
    basket_count = len(basket_list)
    
    phase1_map = basket_rdd.mapPartitions(apriori).flatMap(lambda x: x)
    phase1_reduce = phase1_map.reduceByKey(lambda x, y: x + y).persist()

    phase2_map = phase1_reduce.map(lambda x: son(x[0])).flatMap(lambda x: x)
    phase2_reduce = phase2_map.filter(lambda x: x[1] >= support).map(lambda x: x[0]).collect()

    phase1_output = get_output(phase1_reduce, False)[:-1]
    phase2_output = get_output(phase2_reduce, True)[:-1]

    sc.stop()
    
    output_filepath = f'{output_dir}{dataset_name}_minsup_{support}.txt'
    write_to_file(phase1_output, phase2_output, output_filepath)
    
    print(Fore.BLUE + f'--'*30)
    print(Fore.WHITE + f"| MINSUP:",Fore.GREEN + f"{support}")
    print(Fore.WHITE + f"| Time taken:", Fore.GREEN + f"{round(time.time() - start, 1)} seconds.")
    print(Fore.WHITE + f"| Outputs saved to", Fore.GREEN +  f"{output_filepath}.")
    print(Fore.BLUE + f'--'*30)
    
except Exception as e:
    sc.stop()
    print(Fore.RED + f'Spark Server stopped due to exception.')
    print(Fore.RED + f'Error: {e}')

Running SON Apriori Algorithm on movielens with minsup value: 150



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 04:37:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


------------------------------------------------------------
| MINSUP: 150
| Time taken: 14.2 seconds.
| Outputs saved to ./logs/movielens/movielens_minsup_150.txt.
------------------------------------------------------------


In [7]:
start = time.time()

phase1_candidates = []
support = minsup_values_list[4]

print(Fore.WHITE + f'Running SON Apriori Algorithm on', Fore.GREEN + f'{dataset_name}', 
        Fore.WHITE + f'with minsup value: {support}\n')


spark_configuration = SparkConf().setAppName("cz4032_task2").setMaster('local[*]')
sc = SparkContext(conf=spark_configuration)
sc.setLogLevel("ERROR")

try:
    rdd = sc.textFile(input_filepath)
    

    basket_rdd = rdd.filter(
        lambda x: x != column_names).map(
            lambda x: get_required_bucket(x)
            ).groupByKey().mapValues(set).map(
                lambda x: x[1]
                ).persist()
        
    basket_list = basket_rdd.collect()
    basket_count = len(basket_list)
    
    phase1_map = basket_rdd.mapPartitions(apriori).flatMap(lambda x: x)
    phase1_reduce = phase1_map.reduceByKey(lambda x, y: x + y).persist()

    phase2_map = phase1_reduce.map(lambda x: son(x[0])).flatMap(lambda x: x)
    phase2_reduce = phase2_map.filter(lambda x: x[1] >= support).map(lambda x: x[0]).collect()

    phase1_output = get_output(phase1_reduce, False)[:-1]
    phase2_output = get_output(phase2_reduce, True)[:-1]

    sc.stop()
    
    output_filepath = f'{output_dir}{dataset_name}_minsup_{support}.txt'
    write_to_file(phase1_output, phase2_output, output_filepath)
    
    print(Fore.BLUE + f'--'*30)
    print(Fore.WHITE + f"| MINSUP:",Fore.GREEN + f"{support}")
    print(Fore.WHITE + f"| Time taken:", Fore.GREEN + f"{round(time.time() - start, 1)} seconds.")
    print(Fore.WHITE + f"| Outputs saved to", Fore.GREEN +  f"{output_filepath}.")
    print(Fore.BLUE + f'--'*30)
    
except Exception as e:
    sc.stop()
    print(Fore.RED + f'Spark Server stopped due to exception.')
    print(Fore.RED + f'Error: {e}')

Running SON Apriori Algorithm on movielens with minsup value: 100



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 04:38:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


------------------------------------------------------------
| MINSUP: 100
| Time taken: 13.5 seconds.
| Outputs saved to ./logs/movielens/movielens_minsup_100.txt.
------------------------------------------------------------
